In [3]:
import pygame
from pygame.locals import *
import sys
import numpy as np
import numpy.random as random
import math
import time

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [42]:
lt=[100,700];rt=[700,700];
lb=[100,100];rb=[700,100];
exit=[380,420]

walls=[[lb[0],lb[1],rb[0],rb[1]  ],
       [lb[0],lb[1],lt[0],lt[1]  ],
       [lt[0],lt[1],rt[0],rt[1]  ],
       [rb[0],rb[1],rb[0],exit[0]],
       [rt[0],rt[1],rt[0],exit[1]]]

def distance_agent_to_wall(point,wall):
    p0=np.array(wall[0],wall[1])
    p1=np.array(wall[2],wall[3])
    d=p1-p0
    ymp0=point-p0
    t=np.dot(d,ymp0)/np.dot(d,d)
    if t<=0:
        dist = np.sqrt(np.dot(ymp0,ymp0))
        cross=p0+t*d
    elif t>=1:
        ymp1=point-p1
        dist

In [55]:
for wall in walls:
    print(wall)

[100, 100, 700, 100]
[100, 100, 100, 700]
[100, 700, 700, 700]
[700, 100, 700, 380]
[700, 700, 700, 420]


In [59]:
wall=walls[0]
p0=np.array([wall[0],wall[1]])
p1=np.array([wall[2],wall[3]])
d=p1-p0

print(p0,p1,d)

[100 100] [700 100] [600   0]


In [61]:
class Agent(object):
    def __init__(self,mass,radius,posx,posy,dSpeed):
        
        self.mass = mass 
        self.radius = radius
        
        self.posx = posx
        self.posy = posy
        self.pos = np.array([self.posx, self.posy])
    
        self.aVelocityX = 0 
        self.aVelocityY = 0  
        self.aVelocity = np.array([self.aVelocityX, self.aVelocityY])
    
        self.dest = np.array([random.uniform(100,700),random.uniform(100,700)])
        self.direction = normalize(self.dest - self.pos)
        
        self.dSpeed = dSpeed
        self.dVelocity = self.dSpeed*self.direction
        
        self.acclTime = 0.5 
        self.drivenAcc = (self.dVelocity - self.aVelocity)/self.acclTime
              
        
        self.bodyFactor = 120000
        self.F = 2000
        self.delta = 0.08*50 
        
    def velocity_force(self): 
        deltaV = self.dVelocity - self.aVelocity
        if np.allclose(deltaV, np.zeros(2)):
            deltaV = np.zeros(2)
        return deltaV*self.mass/self.acclTime
    
    
    def f_ij(self, other): 
        r_ij = self.radius + other.radius
        d_ij = np.linalg.norm(self.pos - other.pos)
        e_ij = (self.pos - other.pos)/d_ij
        value = self.F*np.exp((r_ij-d_ij)/(self.delta))*e_ij
        + self.bodyFactor*g(r_ij-d_ij)*e_ij
        return value
    
    def f_ik_wall(self, wall): 
        r_i = self.radius
        d_iw,e_iw = distance_agent_to_wall(self.pos,wall)
        value = -self.F*np.exp((r_i-d_iw)/self.delta)*e_iw 
        + self.bodyFactor*g(r_i-d_iw)*e_iw
        return value
    
    def update_dest(self):
        dist = math.sqrt((self.pos[0]-700)**2 + (self.pos[1]-400)**2)
        if dist < 300:
            self.dest = np.array([700,400])
        else:
            self.dest = np.array([700,400])


In [1]:
    while True:
        # # Finding delta t for this frame
        dt = clock.tick(70)/1000
        roomscreen.fill(background_color)
        
        # draw walls
        for wall in walls:
            start_posw = np.array([wall[0],wall[1]])
            end_posw = np.array([wall[2],wall[3]])
            start_posx = start_posw 
            end_posx = end_posw
            pygame.draw.line(roomscreen, line_color, start_posx, end_posx, 3)
        
        for agent_i in agents:
            aVelocity_force = agent_i.velocity_force()
            people_interaction = 0.0
            wall_interaction = 0.0
        
            for agent_j in agents: 
                if agent_i == agent_j: continue
                people_interaction += agent_i.f_ij(agent_j)
        
            for wall in walls:
                wall_interaction += agent_i.f_ik_wall(wall)
            
            sumForce = aVelocity_force + people_interaction + wall_interaction
            dv_dt = sumForce/agent_i.mass
            agent_i.aVelocity = agent_i.aVelocity + dv_dt*dt 
            agent_i.pos = agent_i.pos + agent_i.aVelocity*dt
            
            # Avoiding disappearing agents   
            if agent_i.pos[0] > 750 or agent_i.pos[0] < 50 or agent_i.pos[1] > 750 or agent_i.pos[1] < 50:
                main()
                sys.exit()
        
        for agent_i in agents:
            # agent_i.time += clock.get_time()/1000 
            start_position = [0, 0]
            start_position[0] = int(agent_i.pos[0])
            start_position[1] = int(agent_i.pos[1])
            
            end_position = [0, 0]
            end_position[0] = int(agent_i.pos[0] + agent_i.aVelocity[0])
            end_position[1] = int(agent_i.pos[1] + agent_i.aVelocity[1])
        
            end_positionDV = [0, 0]
            end_positionDV[0] = int(agent_i.pos[0] + agent_i.dVelocity[0])
            end_positionDV[1] = int(agent_i.pos[1] + agent_i.dVelocity[1])
        
            if start_position[0] > 699 or start_position[0] < 100:
                data_matrix[count+j*nr_agents][2] = count 
                agents.remove(agent_i)
            
            pygame.draw.circle(roomscreen, agent_color, start_position, round(agent_i.radius), 3)
            pygame.draw.line(roomscreen, agent_color, start_position, end_positionDV, 2)
        pygame.display.flip()
    pygame.quit()
main()
#np.savetxt('room1_vis', data_matrix)

NameError: name 'clock' is not defined

In [3]:
import sys, pygame
from pygame.locals import*

width = 1000
height = 500
screen_color = (49, 150, 100)
line_color = (255, 0, 0)

def main():
    screen=pygame.display.set_mode((width,height))
    screen.fill(screen_color)
    
    pygame.draw.line(screen,line_color, (60, 80), (130, 100))
    pygame.display.flip()
    while True:
        for events in pygame.event.get():
            if events.type == QUIT or events.type==pygame.MOUSEBUTTONDOWN:
                sys.exit(0)
main()

SystemExit: 0